In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cd drive/My\ Drive/Colab\ Notebooks
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp drive/My\ Drive/Colab\ Notebooks/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !cd drive/My\ Drive/Colab\ Notebooks
# !kaggle competitions download -c blood-vessel-segmentation --force

In [ ]:
# !unzip blood-vessel-segmentation.zip -d drive/My\ Drive/Colab\ Notebooks/sennet

In [ ]:
!pip install livelossplot

In [ ]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU
from livelossplot.tf_keras import PlotLossesCallback
from tensorflow.keras.models import save_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from math import log2, floor, ceil

#(1706, 1510)
#(1041, 1511)
#(1303, 912)
#(1928, 1928)
#(1303, 912)

In [ ]:
#label is output (image -> label)

# use U-net

# to vừa phải, to quá sẽ overfit
def conv_block(inputs, filters):
    x = tf.keras.layers.Conv2D(filters, 3, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    #x = tf.keras.layers.Dropout(0.5)(x)    #deal with vanishing grad (maybe work?)
    x = tf.keras.layers.Conv2D(filters, 3, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    return x

def downsample_block(inputs, filters):
    x = conv_block(inputs, filters)
    p = tf.keras.layers.MaxPooling2D(2)(x)
    return x, p

def upsample_block(inputs, skip, filters):
    us = tf.keras.layers.Conv2DTranspose(filters, 2, strides=2, padding='same')(inputs)
    concat = tf.keras.layers.Concatenate()([us, skip])
    x = conv_block(concat, filters)
    return x

def unet(input_shape):
    inputs = tf.keras.layers.Input(input_shape)

    # Encoder
    enc1, pool1 = downsample_block(inputs, 64)
    enc2, pool2 = downsample_block(pool1, 128)
    enc3, pool3 = downsample_block(pool2, 256)
    enc4, pool4 = downsample_block(pool3, 512)
    #enc5, pool5 = downsample_block(pool4, 1024)
    #enc6, pool6 = downsample_block(pool5, 2048)

    # Bottleneck
    bottleneck = conv_block(pool4, 1024)

    # Decoder
    #dec6 = upsample_block(bottleneck, enc6, 2048)
    #dec5 = upsample_block(bottleneck, enc5, 1024)
    dec4 = upsample_block(bottleneck, enc4, 512)
    dec3 = upsample_block(dec4, enc3, 256)
    dec2 = upsample_block(dec3, enc2, 128)
    dec1 = upsample_block(dec2, enc1, 64)

    outputs = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(dec1)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = unet((None, None, 1))
# can use some other loss func (dice loss)
# use histogram to manage gradient
optimizer = tf.keras.optimizers.Adam(0.0006)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# cur_dir = "/content/drive/MyDrive/Colab Notebooks/sennet/train/kidney_1_voi"
# label_dir = os.path.join(cur_dir, "labels")

# label = cv2.imread(os.path.join(label_dir, "0000.tif"), cv2.IMREAD_UNCHANGED)
# print(label.shape)
# new_shape = tuple(ceil(k/16) for k in label.shape)
# target_height = new_shape[0]*16
# target_width = new_shape[1]*16
# print(new_shape)
# offset_height = (target_height - label.shape[0]) // 2
# offset_width = (target_width - label.shape[1]) // 2

# label_tf = tf.expand_dims(tf.convert_to_tensor(label, dtype=tf.float32), axis=-1)
# label = tf.image.pad_to_bounding_box(
#     label_tf, offset_height, offset_width, target_height, target_width
# )

# print(label.shape)

In [ ]:
#add early stop
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1) #, start_from_epoch = 5) #val_accuracy makes train accuracy low
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00002)

#read .tiff files
#the image is too large. Considering running just a small set of image each time

# if os.path.exists("/content/drive/MyDrive/Colab Notebooks/sennet/uNet.keras"):
#   model.load_weights("/content/drive/MyDrive/Colab Notebooks/sennet/uNet.keras")

#TODO: after each dir, load weight and run model
#TODO: code chọn random 100 img cùng size mỗi lần để chạy
#will the number of parameter change? no
# do we need to crop?

dirs = os.listdir("/content/drive/MyDrive/Colab Notebooks/sennet/train")
np.random.shuffle(dirs)

trained_imgs = {}
for dir in dirs:
  trained_imgs[dir] = []
# why dont add all img here and then remove?





#TODO: remove dir in dirs when traing all the image inside

# for dirname in os.listdir("/content/drive/MyDrive/Colab Notebooks/sennet/train"):
#     cur_dir = os.path.join("/content/drive/MyDrive/Colab Notebooks/sennet/train", dirname)
#     X = []
#     Y = []
#     i = 0

while len(dirs) > 0:
  chosen_dir = dirs[randint(0, len(dirs) - 1)]
  cur_dir = os.path.join("/content/drive/MyDrive/Colab Notebooks/sennet/train", chosen_dir)

  imgs = os.listdir(os.path.join(cur_dir, "labels"))
  np.random.shuffle(imgs)

  while i < len(imgs):
    X = []
    Y = []
    for k in range (0, 100): #5 for testing, change when run
    # smaller is more prone to overfit
      if i >= len(imgs): break
      label_dir = os.path.join(cur_dir, "labels")
      label = cv2.imread(os.path.join(label_dir, imgs[i]), cv2.IMREAD_UNCHANGED)

      if cur_dir == "/content/drive/MyDrive/Colab Notebooks/sennet/train/kidney_3_dense":
          img_dir = "/content/drive/MyDrive/Colab Notebooks/sennet/train/kidney_3_sparse/images"
      else:
          img_dir = os.path.join(cur_dir, "images")

      img = cv2.imread(os.path.join(img_dir, imgs[i]), cv2.IMREAD_UNCHANGED)
      img = (img - img.min()) / (img.max() - img.min())

      new_shape = tuple(ceil(k/64) for k in img.shape)
      target_height = new_shape[0]*64
      target_width = new_shape[1]*64
      #print(new_shape)
      offset_height = (target_height - label.shape[0]) // 2
      offset_width = (target_width - label.shape[1]) // 2

      img_tf = tf.expand_dims(tf.convert_to_tensor(img, dtype=tf.float32), axis=-1)
      label_tf = tf.expand_dims(tf.convert_to_tensor(label, dtype=tf.float32), axis=-1) / 255
      img = tf.image.pad_to_bounding_box(img_tf, offset_height, offset_width, target_height, target_width)
      label = tf.image.pad_to_bounding_box(label_tf, offset_height, offset_width, target_height, target_width)
      X.append(img)
      Y.append(label)


      #flip
      # img = tf.image.flip_left_right(img)
      # label = tf.image.flip_left_right(label)
      # X.append(img)
      # Y.append(label)

      # img = tf.image.flip_up_down(img)
      # label = tf.image.flip_up_down(label)
      # X.append(img)
      # Y.append(label)

      i += 1

    #optimizer.learning_rate.assign(0.0008)
    callbacks = [PlotLossesCallback(), early_stopping]#, reduce_lr]
    #dont have cross val yet
    model.fit(
        np.array(X), np.array(Y),
        validation_split=.2,
        batch_size=5,     #change this
        epochs=20,       #change this
        verbose=1,
        callbacks=callbacks
    )

    print("Train on: " + cur_dir + ", i = " + str(i))
    file1 = open('/content/drive/MyDrive/Colab Notebooks/sennet/train.txt', 'a')
    file1.write("Train on: " + cur_dir + ", i = " + str(i) + "\n")
    file1.close()

  #TODO: save parameter
  save_model(model, "/content/drive/MyDrive/Colab Notebooks/sennet/uNet.keras")

Read: /content/drive/MyDrive/Colab Notebooks/sennet/train/kidney_1_dense
Epoch 1/20


In [ ]:
# Low validation accuracy - high training accuracy: overfit
# Validation loss increase: overfit
# low training accuracy - high validation accuracy: underfit
# 0 nên dùng model quá to (overfit)

In [ ]:
#output is image, so very much likely to underfit (apply for img seg?)

In [ ]:
#save test img to an array
#change test size to 2048, then after predict crop and change to rle
#TODO: write this
test = []
for dirname in os.listdir("/content/drive/MyDrive/Colab Notebooks/sennet/test"):
  cur_dir = os.path.join("/content/drive/MyDrive/Colab Notebooks/sennet/test", dirname)
  print("Read: " + cur_dir)

  X = []
  Y = []
  i = 0
  dirs = os.listdir(os.path.join(cur_dir, "labels"))

  for k in range (0, 5): #5 for testing, increase when run
    if i >= len(dirs): break
    label_dir = os.path.join(cur_dir, "labels")
    label = cv2.imread(os.path.join(label_dir, dirs[i]), cv2.IMREAD_UNCHANGED)

    if cur_dir == "/content/drive/MyDrive/Colab Notebooks/sennet/train/kidney_3_dense":
        img_dir = "/content/drive/MyDrive/Colab Notebooks/sennet/train/kidney_3_sparse/images"
        img = cv2.imread(os.path.join(img_dir, dirs[i]), cv2.IMREAD_UNCHANGED)
    else:
        img_dir = os.path.join(cur_dir, "images")
        img = cv2.imread(os.path.join(img_dir, dirs[i]), cv2.IMREAD_UNCHANGED)

    img_tf = tf.expand_dims(tf.convert_to_tensor(img, dtype=tf.float32), axis=-1)
    label_tf = tf.expand_dims(tf.convert_to_tensor(label, dtype=tf.float32), axis=-1)
    # img_tf.shape  #return a tuple, use 'for' to access.
    # shape is same for each dir, so calculate new dim at the beginning of the for loop
    #/2 3 lần (/8), chỉ cần làm tròn lên. Nhưng liệu có to quá?
    img = tf.image.resize_with_pad(img_tf, 512, 512)
    label = tf.image.resize_with_pad(label_tf, 512, 512)
    X.append(img)
    Y.append(label)
    i += 1

In [ ]:
#predict test data
p = model.predict(np.array())